<a href="https://colab.research.google.com/github/VuongTuanKhanh/Funix-Capstone-Project/blob/main/notebooks/siammot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls /usr/local/

!python -c "import torch; print(torch.__version__, torch.version.cuda)"

if False:
    !pip uninstall torch torchvision -y
    !pip install torch torchvision

!python -c "import torch; print(torch.__version__, torch.version.cuda)"

bin	   cuda-10.1  games		  lib	       man	  share
cuda	   cuda-11.0  _gcs_config_ops.so  LICENSE.txt  sbin	  src
cuda-10.0  etc	      include		  licensing    setup.cfg  xgboost
1.9.0+cu102 10.2
1.9.0+cu102 10.2


In [ ]:
from IPython.display import clear_output

%cd /content/
!rm -rf apex
!git clone https://github.com/NVIDIA/apex
%cd apex
!CUDA_HOME=/usr/local/cuda-10.0 pip install -v --disable-pip-version-check --no-cache-dir ./

clear_output()
print("apex installed.")

apex installed.


In [ ]:
!pip install ninja yacs cython
clear_output()
print("ninja installed.")

ninja installed.


In [ ]:
# install PyTorch Detection
%cd /content
!rm -rf maskrcnn-benchmark
#!git clone https://github.com/facebookresearch/maskrcnn-benchmark.git
!git clone https://github.com/tiendzung-le/maskrcnn-benchmark.git
%cd maskrcnn-benchmark
!perl -i -pe 's/AT_CHECK/TORCH_CHECK/' $"maskrcnn_benchmark/csrc/cuda"/deform_pool_cuda.cu $"maskrcnn_benchmark/csrc/cuda"/deform_conv_cuda.cu
!python setup.py build develop

from IPython.display import clear_output
clear_output()
print("maskrcnn-benchmark installed")

maskrcnn-benchmark installed


In [ ]:
%cd /content/maskrcnn-benchmark/demo
import sys
sys.path.append("/content/maskrcnn-benchmark") # go to parent dir
import maskrcnn_benchmark

#%load_ext autoreload
#%autoreload 2

/content/maskrcnn-benchmark/demo


Siam_mot

In [ ]:
%cd /content

/content


In [ ]:
import json
import random
import os, sys
from IPython.display import display, clear_output, HTML
from random import randrange, choice
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=25,25
import numpy as np
import seaborn as sns

# Because Life, Universe and Everything!
random.seed(42)

def mdprint(text):
    display({
        'text/markdown': text,
        'text/plain': text
    }, raw=True)

!git clone http://gitlab.aicrowd.com/amazon-prime-air/airborne-detection-starter-kit.git
os.chdir("airborne-detection-starter-kit/data")

Cloning into 'airborne-detection-starter-kit'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 360 (delta 9), reused 8 (delta 3), pack-reused 337
Receiving objects: 100% (360/360), 21.91 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
# Dataset for Airborne Object Tracking Dataset
sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())))
sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())) + "/core")
!pip install -r ../requirements.txt > /dev/null
from core.dataset import Dataset
notebook_path = os.path.dirname(os.path.realpath("__file__"))

local_path = notebook_path + '/part2'
s3_path = 's3://airborne-obj-detection-challenge-training/part2/'
dataset = Dataset(local_path, s3_path)

2021-08-18 06:27:10.715 | INFO     | core.dataset:load_gt:20 - Loading ground truth...
2021-08-18 06:27:10.716 | INFO     | core.file_handler:download_file_if_needed:33 - [download_from_s3] File not found locally, downloading: ImageSets/groundtruth.json


In [ ]:
os.chdir("/content/airborne-detection-starter-kit/")
!ln -s $PWD/data/part2/Images $PWD/data/val

In [ ]:
!pip install timm cityscapesscripts
clear_output()

     |████████████████████████████████| 376 kB 9.0 MB/s 


49 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,623 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 148486 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Updated git hooks.
Git LFS initialized.
Git LFS: (

In [ ]:
%cd /content/airborne-detection-starter-kit
!apt update -qq && apt install git-lfs

!git-lfs install
!git-lfs pull

clear_output()

!pwd
!ls -alh siam-mot/models
!ls -alh siam-mot/configs/dla

/content/airborne-detection-starter-kit
total 92M
drwxr-xr-x 2 root root 4.0K Aug 18 06:27 .
drwxr-xr-x 7 root root 4.0K Aug 18 06:29 ..
-rw-r--r-- 1 root root  92M Aug 18 06:35 DLA-34-FPN_box_track_aot_d4.pth
total 16K
drwxr-xr-x 2 root root 4.0K Aug 18 06:27 .
drwxr-xr-x 3 root root 4.0K Aug 18 06:27 ..
-rw-r--r-- 1 root root 1.8K Aug 18 06:27 DLA_34_FPN_AOT.yaml
-rw-r--r-- 1 root root 1.6K Aug 18 06:27 DLA_34_FPN.yaml


In [ ]:
# Download validation flights
# - e0d815053c1c46cfbd0b586b72718feb
# - ac23cb93c5c242d2b1bf0633fae9b1e6

flight = dataset.get_flight_by_id("e0d815053c1c46cfbd0b586b72718feb")
flight.download()

flight = dataset.get_flight_by_id("ac23cb93c5c242d2b1bf0633fae9b1e6")
flight.download()

In [ ]:
!PYTHONPATH=$PYTHONPATH:/content/airborne-detection-starter-kit/siam-mot python siam_mot_test.py

INFO:maskrcnn_benchmark.utils.model_serialization:base_layer.0.weight                 loaded from base_layer.0.weight                 of shape (16, 3, 7, 7)
INFO:maskrcnn_benchmark.utils.model_serialization:base_layer.1.bias                   loaded from base_layer.1.bias                   of shape (16,)
INFO:maskrcnn_benchmark.utils.model_serialization:base_layer.1.running_mean           loaded from base_layer.1.running_mean           of shape (16,)
INFO:maskrcnn_benchmark.utils.model_serialization:base_layer.1.running_var            loaded from base_layer.1.running_var            of shape (16,)
INFO:maskrcnn_benchmark.utils.model_serialization:base_layer.1.weight                 loaded from base_layer.1.weight                 of shape (16,)
INFO:maskrcnn_benchmark.utils.model_serialization:level0.0.weight                     loaded from level0.0.weight                     of shape (16, 16, 3, 3)
INFO:maskrcnn_benchmark.utils.model_serialization:level0.1.bias                       loa

In [ ]:
# 1.60it/s is quite useful, or 12.28 to run 1200 steps, 
# looks like your model almost fits the time budget
# 800s per flight
# Siam-mot-baseline took around 400s per flight => 0.8 it/s (in colab pro: 3 it / s)
# Can only run 1.8 times x siam-mot-tracker
!ls -alh /content/airborne-detection-starter-kit/data/results/run0

total 100K
drwxr-xr-x 4 root root 4.0K Aug 18 07:09 .
drwxr-xr-x 3 root root 4.0K Aug 18 06:29 ..
drwxr-xr-x 2 root root 4.0K Aug 18 07:03 ac23cb93c5c242d2b1bf0633fae9b1e6
drwxr-xr-x 2 root root 4.0K Aug 18 07:09 e0d815053c1c46cfbd0b586b72718feb
-rw-r--r-- 1 root root  83K Aug 18 07:09 result.json


In [ ]:
# Generate groundtruth.json for relevant flights
def generate_partial_gt():
    flights = ['e0d815053c1c46cfbd0b586b72718feb', 'ac23cb93c5c242d2b1bf0633fae9b1e6']
    gt = json.loads(open("data/part2/ImageSets/groundtruth.json").read())
    for sample in list(gt['samples'].keys()):
        if sample not in flights:
            del gt['samples'][sample]

    with open(("data/evaluation/gt/groundtruth.json"), 'w') as fp:
        json.dump(gt, fp)

# Transfer generated results to metrics codebase bbox format
def convert_and_copy_generated_results_to_metrics_folder():
    flight_results = json.loads(open("data/results/run0/result.json").read())
    for i in range(len(flight_results)):
        for j in range(len(flight_results[i]['detections'])):
            x = flight_results[i]['detections'][j]['x']
            y = flight_results[i]['detections'][j]['y']
            w = flight_results[i]['detections'][j]['w'] - x
            h = flight_results[i]['detections'][j]['h'] - y


            flight_results[i]['detections'][j]['x'] = x + w/2
            flight_results[i]['detections'][j]['y'] = y + h/2
            flight_results[i]['detections'][j]['w'] = w
            flight_results[i]['detections'][j]['h'] = h

    with open("data/evaluation/result/result.json", 'w') as fp:
        json.dump(flight_results, fp)

!pwd
!rm -rf data/evaluation/
!mkdir -p data/evaluation/gt
!mkdir -p data/evaluation/result

generate_partial_gt()
convert_and_copy_generated_results_to_metrics_folder()
!python core/metrics/run_airborne_metrics.py --dataset-folder data/evaluation/gt --results-folder data/evaluation/result --summaries-folder data/evaluation/summaries


/content/airborne-detection-starter-kit
2021-08-18 07:17:13,410:INFO:run_airborne_metrics.py:160 Encounter ground truth: data/evaluation/gt/groundtruth_with_encounters_maxRange700_maxGap3_minEncLen30.csv
2021-08-18 07:17:13,411:INFO:calculate_encounters.py:83 Asserting data/evaluation/gt/groundtruth.json format
2021-08-18 07:17:13,411:INFO:pandas_utils.py:87 Reading ground truth
2021-08-18 07:17:13,411:INFO:pandas_utils.py:61 Reading provided data/evaluation/gt/groundtruth.json
2021-08-18 07:17:13,411:INFO:pandas_utils.py:68 Loading .json
2021-08-18 07:17:13,420:INFO:pandas_utils.py:75 Normalizing json. This operation is time consuming. The result .csv will be saved Please consider providing .csv file next time
2021-08-18 07:17:13,514:INFO:calculate_encounters.py:268 Saving groundtruth in .csv format, please use .csv in the future
2021-08-18 07:17:13,542:INFO:calculate_encounters.py:273 Filtering ground truth to get intruders in the specified range <= 700m.
2021-08-18 07:17:13,571:INFO

In [ ]:
!cat data/evaluation/summaries/*_for_ranking.csv

cat: 'data/evaluation/summaries/*_for_ranking.csv': No such file or directory
